In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1=nn.Sequential(nn.Conv2d(1, 32, 7, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            #nn.Conv2d(32, 32, 3, padding=1),
            #nn.ReLU(),
            #nn.BatchNorm2d(32),
            #nn.Conv2d(32, 32, 3, stride=2, padding=1),
            #nn.ReLU(),
            #nn.BatchNorm2d(32),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25) )
    self.conv2=nn.Sequential(nn.Conv2d(32, 64, 1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            #nn.Conv2d(64, 64, 1, padding=1),
            #nn.ReLU(),
            #nn.BatchNorm2d(64),
            #nn.Conv2d(64, 64, 1, stride=2, padding=1),
            #nn.ReLU(),
            #nn.BatchNorm2d(64),
            #nn.MaxPool2d(2, 2),
            nn.Dropout(0.25))
    self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, 1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            #nn.Conv2d(128, 128, 1, padding=1),
            #nn.ReLU(),
            #nn.Conv2d(128, 128, 1, stride=2,padding=1),
            #nn.ReLU(),
            #nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
            )
    self.conv4 = nn.Sequential(
            nn.Conv2d(128, 256, 1,padding=1),
            nn.ReLU(),
            #nn.Conv2d(256, 256, 3, padding=1),
            #nn.ReLU(),
            #nn.Conv2d(256, 256, 3, stride=2,padding=1),
            #nn.ReLU(),
            nn.BatchNorm2d(256),
            #nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
            )
    self.conv5 = nn.Sequential(
            nn.Conv2d(256, 512,11, padding=1),
            nn.ReLU(),
            #nn.Conv2d(512, 512, 3, padding=1),
            #nn.ReLU(),
            #nn.Conv2d(512, 512, 3),
            #nn.ReLU(),
            #nn.BatchNorm2d(512),
            #nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
            )
    self.conv6 = nn.Sequential(
            nn.Conv2d(512, 1024, 3),
            #nn.ReLU(),
            #nn.BatchNorm2d(1024),
            #nn.MaxPool2d(2, 2),
            #nn.Dropout(0.25)
            )
    self.conv7 = nn.Sequential(
            nn.Conv2d(1024, 10, 3),
            #nn.ReLU(),
            #nn.BatchNorm2d(10),
            #nn.MaxPool2d(2, 2),
            #nn.Dropout(0.25)
            )
    self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=10))
  def forward(self,x):
      x = self.conv1(x)
      x = self.conv2(x)
      x = self.conv3(x)
      #x = self.conv4(x)
      #x = self.conv5(x)
      #x = self.conv6(x)
      #x = self.conv7(x)
      x = self.gap(x)
      x = x.view(x.size(0), -1)
      x = F.log_softmax(x, dim=1)
      return x

    
    

    
  
  
  
        


In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model,input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 24, 24]           1,600
              ReLU-2           [-1, 32, 24, 24]               0
       BatchNorm2d-3           [-1, 32, 24, 24]              64
         MaxPool2d-4           [-1, 32, 12, 12]               0
           Dropout-5           [-1, 32, 12, 12]               0
            Conv2d-6           [-1, 64, 14, 14]           2,112
              ReLU-7           [-1, 64, 14, 14]               0
       BatchNorm2d-8           [-1, 64, 14, 14]             128
           Dropout-9           [-1, 64, 14, 14]               0
           Conv2d-10          [-1, 128, 16, 16]           8,320
             ReLU-11          [-1, 128, 16, 16]               0
      BatchNorm2d-12          [-1, 128, 16, 16]             256
          Dropout-13          [-1, 128, 16, 16]               0
        AvgPool2d-14            [-1, 12

In [ ]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 18):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 1.1945, Accuracy: 8198/10000 (82%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.6038, Accuracy: 9041/10000 (90%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.1718, Accuracy: 9580/10000 (96%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1641, Accuracy: 9599/10000 (96%)



loss=0.3676603138446808 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 33.97it/s]



Test set: Average loss: 0.1632, Accuracy: 9579/10000 (96%)

